In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('dannys_diner.sqlite3') 

### Question 1: How much did each customer spend?

In [3]:
sql = ('''
       SELECT customer_id, SUM(price) AS spend
       FROM sales
       INNER JOIN menu
         ON sales.product_id = menu.product_id
       GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  spend
0           A     76
1           B     74
2           C     36


### Question 2: How many days has each customer visited the restaurant?

In [4]:
sql = ('''
       SELECT customer_id, COUNT(DISTINCT order_date) AS visit_days
       FROM sales
       GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  visit_days
0           A           4
1           B           6
2           C           2


### Question 3: What was the first item from the menu purchased by each customer?

In [5]:
sql = ('''
       WITH product_rn AS (
	        SELECT *,
		           ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY order_date) as row_num
            FROM sales)

       SELECT customer_id, product_name as first_product
       FROM product_rn
       INNER JOIN menu
         ON product_rn.product_id = menu.product_id
        WHERE row_num = 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id first_product
0           A         sushi
1           B         curry
2           C         ramen


### Question 4: What is the most purchased item on the menu and how many times was it purchased by all customers?

In [6]:
sql = ('''
       SELECT product_name, COUNT(*) AS purchases
       FROM sales
       INNER JOIN menu
         ON sales.product_id = menu.product_id
       GROUP BY product_name
       ORDER BY purchases DESC
       LIMIT 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  product_name  purchases
0        ramen          8


### Question 5: Which item was the most popular for each customer?

In [7]:
sql = ('''
       WITH most_ordered AS(
         SELECT customer_id,
                product_id,
                COUNT(*) AS purchases,
                RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(*) DESC) AS rank
	     FROM sales
	     GROUP BY customer_id, product_id)

    SELECT customer_id,
        product_name,
        purchases
    FROM most_ordered
    INNER JOIN menu
        ON most_ordered.product_id = menu.product_id
    WHERE rank = 1
    ORDER BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id product_name  purchases
0           A        ramen          3
1           B        sushi          2
2           B        curry          2
3           B        ramen          2
4           C        ramen          3


### Question 6: Which item was purchased first by the customer after they became a member?

In [8]:
sql = ('''
       WITH after_join 
            AS (SELECT *,
                    RANK() OVER(PARTITION BY sales.customer_id ORDER BY order_date) AS ranking
                FROM members
                LEFT JOIN sales
                    ON sales.customer_id = members.customer_id
                LEFT JOIN menu
                    ON sales.product_id = menu.product_id
                WHERE order_date >= (SELECT join_date FROM members)
                )

        SELECT customer_id,
               product_name
        FROM after_join
        WHERE ranking = 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id product_name
0           A        curry
1           B        sushi


### Question 7: Which item was purchased just before the customer became a member?
Note: As customer A purchased two items simultaneously, it's assumed both are to be included.

In [9]:
sql = ('''
       WITH after_join 
        	AS (SELECT *,
					RANK() OVER(PARTITION BY sales.customer_id ORDER BY order_date DESC) AS ranking
				FROM members
				LEFT JOIN sales
					ON sales.customer_id = members.customer_id
				LEFT JOIN menu
					ON sales.product_id = menu.product_id
				WHERE order_date < (SELECT join_date FROM members)
				)

		SELECT customer_id,
			product_name
		FROM after_join
		WHERE ranking = 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id product_name
0           A        sushi
1           A        curry
2           B        sushi


### Question 8: What is the total items and amount spent for each member before they became a member?

In [11]:
sql = ('''
        WITH after_join 
            AS (SELECT sales.customer_id, product_name, price
                FROM members
                LEFT JOIN sales
                    ON sales.customer_id = members.customer_id
                LEFT JOIN menu
                    ON sales.product_id = menu.product_id
                WHERE order_date < (SELECT join_date FROM members)
                )

        SELECT customer_id,
               SUM(price) AS total_spend
        FROM after_join
        GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  total_spend
0           A           25
1           B           40


### Question 9: If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

Note: It's assumed that only members receive points, and only from all purchases from their joining date onwards.

In [12]:
sql = ('''
        WITH after_join 
		    AS (SELECT sales.customer_id,
                       price,
                       sales.product_id
                FROM members
                LEFT JOIN sales
                    ON sales.customer_id = members.customer_id
                LEFT JOIN menu
                    ON sales.product_id = menu.product_id
                WHERE order_date >= (SELECT join_date FROM members)
                )

SELECT customer_id,
	   SUM(CASE
		   WHEN product_id = 1 THEN price * 20
		   ELSE price * 10
		   END) AS points
FROM after_join
GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  points
0           A     510
1           B     440
